# QMUL SBM PHD Python Workshop - Part 4

In this session, we will first learn the basics of Pandas, the Python package for working with tabular data. We will undertake data pre-processing, EDA, and data visualisation. We will then move on to a bit more advanced topics and look at parsing documents with CSS selectors and  expressions.

# Working with data using Pandas

Consider that you have just started working as an analyst at a film production company and your job involves analysing the market trends in the filming industry. As a data source on individual firms and audience preferences, you download the datasets on IMDB https://datasets.imdbws.com/ (download title.basics.tsv.gz and title.ratings.tsv.gz). Your first objective is to clean data in the individual datasets and form an integrated dataset of movies produced in the past two decades, involving Title, Genre, Year, Runtime (Minutes), IMDB Rating, and Number of Votes by merging the two datasets.

## Organising your workspace

To work with data files, we must first ensure that they are in our working directory. There are multiple ways of dealing with the workspace. Here, we use the `os` package. In the below, we change the path to the directory in which we have the data files in our local drive. For this, we can use the shell commands by importing the `os` package (an alternative is passing the commands to the Shell by using an exclamation mark !). The function `os.getcwd()` returns the current working directory, while `os.chdir()` changes the working directory. The `os.path` module contains functions to work with path names in a way that is robust across operating systems (Windows, MacOS, and Linux).

In [ ]:
#import os
import os

# Print your current working directory
print(__)

# assign your home address to the variable HOME. The expanduser function is used to replace ~ with the home
HOME = __(__)

# Locate the folder in which you saved the data and create a path by joining them
# In my case, my HOME is "C:\Users\Guven" and my files are under 
# C:\Users\guven\Documents\PhD workshop\data
PROJECT_DIR = __(__)

# Change to the folder in which the dataset is located
__(__)

# Print your current working directory
print(__)

## Reading data with Pandas

The first step of data analysis is reading data from files. Pandas library provides several functions for reading data from different types of files, including comma or tab separated files. We first read the data from the `title.basics.tsv.gz` file, which includes the following data fields:  
- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
- endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
- runtimeMinutes – primary runtime of the title, in minutes
- genres (string array) – includes up to three genres associated with the title.

We now read the data by the Pandas `read_csv` function, which takes the file path and `sep` (separator, which is `\t` for tab in our data that is tab-seperated). This returns a `dataframe` object which includes different observations (films) in the rows, and features in the columns. Each row is identifed by its index value very much like Python dictionaries.

In [ ]:
# Import pandas library (convention: as pd)
__

# Read the data from title_basics into the dataframe movies_df
__ = __(__, sep=__, low_memory=False)

## Data preprocessing

We shall now explore the data set. For this, we can use the `head()` and `tail()` methods of the dataframe, which display the top or bottom rows, respectively.

In [ ]:
# Display the top 10 rows of movies_df
__

The column `tconst` is the unique identifier for the record. Hence, we would like to set it as the index by the `set_index()` method. The default behaviour of Pandas objects is not to mutate the original data frame and to return a new dataframe. To overwrite the orginal dataframe, you should pass the argument `inplace=True`. Passing the argument `drop=True` leads to dropping the column used for setting the index.

In [ ]:
# Set the index to the tconst column and drop the column
movies_df.__(__, drop=__, __)

# print the tail (last 5)
__

The `info` method provides key information on the data frame, including names and types of variables, data shape (number of rows and columns), and memory. As you can see below, there are 9616165 titles recorded on IMDB.

In [ ]:
# Show info about the dataframe
movies_df.__()

We can now move to data cleaning. First, let's remove duplicate entries, if any, by the `drop_duplicates` method.

In [ ]:
# What is the initial number of rows?
# Hint: you can use the shape attribute as in Numpy arrays
N1 = movies_df.__

# Drop the duplicates in-place
__

# Print the number of rows that have been dropped
__

We can obtain a view of one of the columns by using square brackets [], which keeps the association with the index and returns a Series. Since it is only a view of a part of the original dataframe, if we make any changes, it applies to the original dataframe.

In [ ]:
# Return a view of the "titleType" series
__

Not all of the columns are relevant. Especially if you are working with big data sets, it is beneficial to drop the irrelevant variables directly. You need to identify these variables based on your research questions and initial exploration of the data set. You can use the `drop` method with the argument `axis = 1` to drop the columns, for which you pass the names. You can obtain the columns by using the `columns` attribute of the dataframe.

In [ ]:
# Drop the columns ["originalTitle", "isAdult", "endYear"] 
__

# Check the remaining columns by the columns attribute
__

We need to treat string and numerical variables separately and handle the missing values properly. As an example, we shall first check how many numeric entries the `runtimeMinutes` has. For this, you can call the `pandas.Series.str` functions (`isnumeric` to check whether it is a number).

In [ ]:
# Print the number of entries 
print(__)
    
# Print the number of numeric entries of the runtimeMinutes column
print(__)

## DataType Conversion and Missing Values

The missing values in this data set are encoded as `\N`, which we replace with `Nan` to be properly handled by pandas. We use the `replace()` method to replace a given value with a desired value, in our case `\\N` with `Nan`. 

Note that all variables are currently held as `object` type, which is used when there is mixed datatype and for strings. As we will see, Pandas cannot infer the correct data types in this case due to missing values being recorded as string. We want to ultimately cast to the following data types:
- 'titleType': 'string'
- 'primaryTitle': 'string'
- 'startYear': 'Int64'
- 'runtimeMinutes': 'float64'
- 'genres': 'string'

In [ ]:
import numpy as np

# Replace the missing value place holder \\N with np.nan (missing value indicator)
movies_df.__('\\N', __, __)

We shall now check whether the numeric variables truely hold numeric entries. 

In [ ]:
# Print the number of rows in the dataframe and the number of rows with numeric entries
# for the 'startYear' column
print(__, __, __)

# for the 'runtimeMinutes' column
print(__, __, __)

Since we see a mismatch in the 'runtimeMinutes' (10 entries neither missing nor numeric), let's inspect the column and find the source of the problem. For this we will select the rows where the entry is not numeric passing a boolean array. If you have a dataframe `X` and you select the rows where the column `c1` satisfies a certain condition (say is negative) by `X[X['c1'] < 0]`. 

In [ ]:
# print the rows of the dataframe in which the `runTimeMinutes` is not numeric
movies_df[__].head(20)

In [ ]:
cond1 = (movies_df.runtimeMinutes.str.isnumeric() == False)

# Set the genres to the info in rumtime minutes
__

# Replace non-numeric runtimeMinutes with missing values
__

We are now ready to correct the data types by using the method `astype()` to which we pass a dictionary of data types. Note that we cast 'startYear' as 'float64' first and then to 'Int64' because the 'object' type can be converted to float64 but not Int64 when there are missing values. A work-around is to first convert to float64 and then to int 64.

In [ ]:
# Column dataypes
column_types = {'titleType': 'string', 'primaryTitle': 'string', 'startYear': 'float64', 
                'runtimeMinutes': 'float64', 'genres': 'string'}
# Convert all data types using the dictionary
movies_df = movies_df.__(__) 

# Correct the data type for startYear
movies_df['startYear'] = movies_df['startYear'].__

# Check the variables and data types
movies_df.__

We shall now inspect missing values. You can use the `isna()` method to obtain a data frame which holds a value `True` for cells where the data is missing.

In [ ]:
# Create a Dataframe nullVals that holds an indicator of where values are missing
nullVals = movies_df.__

# print the the number of missing values for each column
nullVals.__

We have only a handful of missing values in the 'Primary Title'. We will exclude those without title.  For this, we use the `dropna()` method. You can specify when to drop, i.e. in which column when there is a missing value, by specifying the `subset` argument.

In [ ]:
# Drop rows if they do not have a title
movies_df.__

 Let's drop cells if both `runtimeMinutes` and `genres` are missing. For this, you can use the `how='all'` argument of the `dropna()` method.

In [ ]:
# If both runtimeMinutes and genres are missing, drop that row
movies_df.__(__=['runtimeMinutes', 'genres'], __, __)

# Create a new data_frame by selecting only movies (titleType = 'movie') that were produced after 2000
movies_selected_df = __

We now drop the 'titleType' field, which is now always movie, hence not needed, rename the variables for convenience, and sort according to a given variable. The `rename()` method takes a dictionary as the columns arguments in the format {var1_old_name: var1_new_name, var2_old_name: var2_new_name}. We then sort the data in the ascending order of year by using the `sort_values()` method by setting the keyword argument `ascending=False`.

In [ ]:
# Drop the column "titleType"
movies_selected_df.__(__, axis = __, inplace = True)

# Rename the variables
movies_selected_df.__(columns = {'primaryTitle': 'movie', 
                            'startYear': 'year', 
                            'runtimeMinutes': 'minutes'}, 
                      inplace = True)

# sort the dataframe wrt year in-place
movies_selected_df.__ 

# show the head of the dataframe
movies_selected_df.__

We can now impute the `minutes` by the median `minutes`. For this, we can use the `fillna()` method with the median as the positional argument.

In [ ]:
# Impute any missing values in the minutes column by its median
movies_selected_df['minutes'] = __

# show the head of the dataframe
movies_selected_df.head()

### DataFrame Merging
We shall now merge the user ratings data with the movie dataframe. For this, we shall first read the title.ratings.tsv.gz dataset to a dataframe as we did for the first data set. Here, we will directly specify the index column, which is `tconst` as for the movie dataframe. This is done by passing the argument `index_col = 'tconst'`. We also set the data types by setting the `dtype` to corresponding data type.   

In [ ]:
# Read the dataset title_ratings.tsv to a dataframe and set the index and the data types
ratings_df = __("title.ratings.tsv.gz", __ = "tconst", 
                         __ = {'averageRating': 'float64', 'numVotes': 'Int64'}, 
                         sep = '\t')

#Show top rows
ratings_df.head()

As you can see, it associates the same identifier `tconst` with the `averageRating` and `numVotes` variables.

Pandas `merge` method allows merging a dataset with another dataset. Here, we use the index `tconst` for matching the two dataframes, by setting the arguments `left_index = True` and `right_index = True`. This means that if two rows in the two datasets have the same index value, they belong to the same entity (movie). The argument `how = 'inner'` specifies that all rows of the left and right frames should match. If there are non-matching indices in either, those rows are excluded.

In [ ]:
# Inner-Merge the ratings_df with movies_selected_df (key should exist in both)
final_movies_df = movies_selected_df.__(__, how = __, left_index = __, right_index = __)

# Show the head of the data frame
final_movies_df.head()

In [ ]:
# Check the number of entries
__

## EDA

We shall now look at the descriptive statistics for numeric variables by using the `describe` method:

In [ ]:
# Check descriptive statistics
final_movies_df.__

There seems to be some movies with very low number of votes. Hence, we shall slice only to those movies with at least 10000 votes. We shall then have a quick look at the top 10 movies with highest `averageRating`.

In [ ]:
# Choose only those movies with at least 10000 votes
final_movies_df = __

# Display the top 10 movies in terms of averageRating (if equal, more votes first)
final_movies_df.__(__, ascending = __).__

We look at the correlation between numerical variables by using the `corr()` method.

In [ ]:
# Check the correlation coefficients 
final_movies_df.__

There are some positive correlations between the number of votes, average rating, and the minutes. However, the correlation coefficient is a linear measure of assocation and it does not mean causation. We shall look at scatterplots. 

Matplotlib is the main visualisation package in Python and standard plots are directly implemented as methods of dataframes. Hence, you can call the `plot` method directly from a dataframe object. Here, we specify the plot type by `kind = 'scatter'`. We provide the x and y axes and the title of the plot.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Scatter plot between numVotes and averageRating
__.plot(kind = __, x =__, y = __, title =__, figsize=(10,8))

We can see that the variation in averageRating decreases with numVotes, similar to central limit theorem as $SE(\bar{x}) = \frac{\sigma}{\sqrt{n}}$ if $n$ individual viewer ratings are independent and randomly sampled. 

We shall now plot and inspect boxplots to see how ratings evolve over time. We can directly plot a boxplot using the `boxplot` method, specifying the column for which to plot the boxplot. The optional argument `by = 'year'` specifies that we want to plot the boxplots separately for different values of the variable `year`.

In [ ]:
# Create a box plot of averageRating by year
final_movies_df.__(column = __, by = __, rot = 90, figsize=(10,8))

The ratings look pretty stable over time.

We shall finally look at the association between the genre and the movie rating. We must consider that each movie can belong to multiple genres. We first start by identifying the different genres. 

In [ ]:
# The cat method concatenates all entries in a column using the specified seperator (here ',')
genres = final_movies_df[__].__.cat(sep = __) 

# This returns a long str of individual genres. To obtain a set of genres, we first
# split from ',' to a list and then remove duplicates by the set() constructor
genres = __

# Write a dictionary comprehension that provides the average score for each genre
avg_ratings = __

#Show average ratings per genre
avg_ratings